For my solo exploration, I want to look at different countries and compare there current consumption (past 5 years) of electric power in thousands of kW per year. I then want to produce the amount of uranium required to power these needs vs coal or gas.    

https://world-nuclear.org/information-library/nuclear-fuel-cycle/introduction/nuclear-fuel-cycle-overview.aspx#:~:text=Typically%2C%20some%2044%20million%20kilowatt,one%20tonne%20of%20natural%20uranium.

This is a link to the reference for how much electricity comes from one ton of natural uranium. 

44 million kW-hr = 1 ton U
44 million kW-hr = 20,000 tons Coal
